In [1]:
import numpy as np
import pandas as pd

import datetime
import warnings
warnings.filterwarnings('ignore')
import sys
if not sys.warnoptions:
    warnings.simplefilter('ignore')
np.random.seed(42)

# Preprocessing & models
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

In [2]:
cust = pd.read_csv("C:/Projects/Automobile-Sales-Logistics/Data/Cleaned-Data/cust_output.csv")

In [3]:
rfm = cust.copy()

# -----------------------------------------
# R, F, M ranks were made to avoid qcut errors.
# -----------------------------------------

# Recency: lower is better → high score to best one
r_rank = rfm['recency_days'].rank(method='first', ascending=True)
rfm['R_score'] = pd.qcut(r_rank, 4, labels=[4, 3, 2, 1])

# Frequency: higher is better
f_rank = rfm['frequency_orders'].rank(method='first', ascending=False)
rfm['F_score'] = pd.qcut(f_rank, 4, labels=[1, 2, 3, 4])

# Monetary: higher is better
m_rank = rfm['monetary_sales'].rank(method='first', ascending=False)
rfm['M_score'] = pd.qcut(m_rank, 4, labels=[1, 2, 3, 4])

# Convert to int and total RFM score
rfm[['R_score', 'F_score', 'M_score']] = rfm[['R_score', 'F_score', 'M_score']].astype(int)
rfm['RFM_score'] = rfm[['R_score', 'F_score', 'M_score']].sum(axis=1)    # Why summed up why not concatenated as a string ?

# -----------------------------------------
# Value Tier (High / Mid / Low)
# -----------------------------------------
# Min RFM = 4, Max ≈ 10 (on this data)
conditions = [
    (rfm['RFM_score'] >= 9),
    (rfm['RFM_score'].between(6, 8)),
    (rfm['RFM_score'] <= 5)
]
choices = ['High Value', 'Mid Value', 'Low Value']

rfm['value_tier'] = np.select(conditions, choices, default='Mid Value')   # What does np do, I mean for series it is helpful, but what is main function.

rfm[['customername', 'R_score', 'F_score', 'M_score', 'RFM_score', 'value_tier']].head()


,customername,R_score,F_score,M_score,RFM_score,value_tier
0,"AV Stores, Co.",2,1,1,4,Low Value
1,Alpha Cognac,4,1,3,8,Mid Value
2,Amica Models & Co.,1,3,2,6,Mid Value
3,"Anna's Decorations, Ltd",3,1,1,5,Low Value
4,Atelier graphique,2,2,4,8,Mid Value


In [4]:
# -----------------------------------------
# Shipping Reliability Buckets
# -----------------------------------------
# Thresholds:
# 0–0.70   → Critical
# 0.70–0.85 → Unreliable
# 0.85–0.95 → Minor Issues
# 0.95–1.0+ → Reliable
# -----------------------------------------

bins = [0, 0.70, 0.85, 0.95, 1.01]
labels = ['Critical', 'Unreliable', 'Minor Issues', 'Reliable']

rfm['ship_bucket'] = pd.cut(
    rfm['shipping_reliability'],
    bins=bins,
    labels=labels,
    include_lowest=True
)

rfm[['customername', 'shipping_reliability', 'ship_bucket']].head()


,customername,shipping_reliability,ship_bucket
0,"AV Stores, Co.",1.0,Reliable
1,Alpha Cognac,1.0,Reliable
2,Amica Models & Co.,1.0,Reliable
3,"Anna's Decorations, Ltd",1.0,Reliable
4,Atelier graphique,1.0,Reliable


In [5]:
# -----------------------------------------
# Composite segment: Value Tier + Shipping Risk
# -----------------------------------------

cond_vs = [
    # High value + bad shipping  → risk
    (rfm['value_tier'] == 'High Value') & rfm['ship_bucket'].isin(['Critical', 'Unreliable']),
    # High value + stable shipping
    (rfm['value_tier'] == 'High Value') & rfm['ship_bucket'].isin(['Minor Issues', 'Reliable']),

    # Mid value + bad shipping
    (rfm['value_tier'] == 'Mid Value') & rfm['ship_bucket'].isin(['Critical', 'Unreliable']),
    # Mid value + stable
    (rfm['value_tier'] == 'Mid Value') & rfm['ship_bucket'].isin(['Minor Issues', 'Reliable']),

    # Low value + bad shipping → low priority
    (rfm['value_tier'] == 'Low Value') & rfm['ship_bucket'].isin(['Critical', 'Unreliable']),
    # Low value + stable
    (rfm['value_tier'] == 'Low Value') & rfm['ship_bucket'].isin(['Minor Issues', 'Reliable']),
]

choices_vs = [
    'High Value – Fulfilment Risk',
    'High Value – Stable',

    'Mid Value – Fulfilment Risk',
    'Mid Value – Stable',

    'Low Value – Not Priority',
    'Low Value – Stable/Upsell'
]

rfm['value_ship_segment'] = np.select(cond_vs, choices_vs, default='Other')

rfm[['customername', 'value_tier', 'ship_bucket', 'value_ship_segment']].head()

,customername,value_tier,ship_bucket,value_ship_segment
0,"AV Stores, Co.",Low Value,Reliable,Low Value – Stable/Upsell
1,Alpha Cognac,Mid Value,Reliable,Mid Value – Stable
2,Amica Models & Co.,Mid Value,Reliable,Mid Value – Stable
3,"Anna's Decorations, Ltd",Low Value,Reliable,Low Value – Stable/Upsell
4,Atelier graphique,Mid Value,Reliable,Mid Value – Stable


In [6]:
# -----------------------------------------
# Features for clustering
# -----------------------------------------

from sklearn.preprocessing import MinMaxScaler



features_for_cluster = [
    'recency_days',
    'frequency_orders',
    'monetary_sales',
    'shipping_reliability',
    'n_unique_products'
]

X = rfm[features_for_cluster].copy()

# MinMax scaling (0–1, clustering friendly)
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# -----------------------------------------
# K-Means (4 clusters)    # how number of clusters were decided here ?
# -----------------------------------------
kmeans = KMeans(
    n_clusters=4,
    random_state=42,
    n_init=10
)
rfm['cluster_kmeans'] = kmeans.fit_predict(X_scaled)

# Cluster-wise behaviour summary
cluster_summary = (
    rfm.groupby('cluster_kmeans')[features_for_cluster + ['RFM_score']]
       .mean()
       .round(1)
)

print(cluster_summary)


                recency_days  frequency_orders  monetary_sales  \
cluster_kmeans                                                   
0                      164.8               3.0         94611.6   
1                       68.8               3.5        113690.5   
2                        2.0              21.5        783576.1   
3                      444.3               2.1         66861.9   

                shipping_reliability  n_unique_products  RFM_score  
cluster_kmeans                                                      
0                                1.0               24.2        7.4  
1                                0.6               29.5        7.6  
2                                0.9               91.5        6.0  
3                                1.0               18.1        8.1  


In [7]:
cluster_name_map = {
    0: "Core Stable Buyers",
    1: "Value Buyers with Shipping Issues",
    2: "Dormant / Aging Customers",
    3: "Ultra VIP Heavy Buyers"
}
rfm['cluster_label'] = rfm['cluster_kmeans'].map(cluster_name_map)


In [8]:
# Revenue by Value Tier
kpi_value_revenue = (
    rfm.groupby('value_tier')['monetary_sales']
       .agg(['count', 'sum', 'mean'])
       .rename(columns={'count': 'n_customers', 'sum': 'total_revenue', 'mean': 'avg_revenue'})
       .sort_values('total_revenue', ascending=False)
)

# Revenue by Value × Shipping segment
kpi_vs_revenue = (
    rfm.groupby('value_ship_segment')['monetary_sales']
       .agg(['count', 'sum', 'mean'])
       .rename(columns={'count': 'n_customers', 'sum': 'total_revenue', 'mean': 'avg_revenue'})
       .sort_values('total_revenue', ascending=False)
)

# Shipping bucket distribution
kpi_ship = (
    rfm['ship_bucket'].value_counts()
       .rename('n_customers')
       .to_frame()
)
kpi_ship['percent'] = (kpi_ship['n_customers'] / len(rfm) * 100).round(1)

print("Value Tier Revenue:\n", kpi_value_revenue)
print("\nValue × Shipping Revenue:\n", kpi_vs_revenue)
print("\nShipping Reliability Distribution:\n", kpi_ship)


Value Tier Revenue:
             n_customers  total_revenue    avg_revenue
value_tier                                           
Mid Value            51     6417402.84  125831.428235
High Value           27     1682711.39   62322.644074
Low Value            11     1660107.48  150918.861818

Value × Shipping Revenue:
                               n_customers  total_revenue    avg_revenue
value_ship_segment                                                     
Mid Value – Stable                     43     4553784.81  105901.972326
Mid Value – Fulfilment Risk             8     1863618.03  232952.253750
Low Value – Stable/Upsell              10     1496038.04  149603.804000
High Value – Stable                    21     1206437.17   57449.389048
High Value – Fulfilment Risk            6      476274.22   79379.036667
Low Value – Not Priority                1      164069.44  164069.440000

Shipping Reliability Distribution:
               n_customers  percent
ship_bucket                      

In [9]:
rfm.to_csv("C:/Projects/Automobile-Sales-Logistics/Data/Cleaned-Data/rfm_output.csv", index=False)